In [1]:
import gensim
import pandas as pd
import numpy as np
from scipy import spatial

In [2]:
df = pd.read_csv("pre_ann.csv", index_col=0)
df

,class,word,sentence1,sentence2
0,NaN,kap,podaš komu na obodu ali ne. Sploh vas bo kap...,Majico in kapo prejmejo:
1,NaN,kap,skorumpiran preprodajalec orožja? woe is us. ...,", da sem prvič in zadnjič nosil peterokrako z..."
2,NaN,kap,izdelki njegovih rok v rojstni hiši. Umrl je ...,"s seboj krizopraz. Nesrečnežu, ki ga je ohrom..."
3,NaN,kap,"odkrili, kako lahko zaradi njih nastane srčni...",", čeprav je bilo zunaj borih trideset stopinj..."
4,NaN,kap,to sploh nima veze. Ženska je res carica. Ka...,"ne bi bilo v sobi. “Z dežja pod kap , hm? In..."
...,...,...,...,...
4995,NaN,vila,"minut vožnje od Opatije, je na ekskluzivni lo...",". Dolge kolone avtomobilov, ki so se konec te..."
4996,NaN,vila,2. Je pravljično lepa vila v resnici Darkov...,"IZ KATEREGA JE MOLČE SNEŽILO, SE JE RAZPENJAL..."
4997,NaN,vila,", za najem stavbe NPU, Hypo banko, gradnjo v...",dokaj na hitro in brez možnosti podaljšanja p...
4998,NaN,vila,Vila pod hollywoodskim znakom,gospa Delopst pa da je prišla pred nje tudi z...


In [3]:
import string

def remove_punc(text):
    return "".join([char for char in text if char not in string.punctuation])

df.sentence1 = df.sentence1.apply(lambda x: x.lower())
df.sentence1 = df.sentence1.apply(lambda x: " ".join(remove_punc(x).split()))
df.sentence2 = df.sentence2.apply(lambda x: x.lower())
df.sentence2 = df.sentence2.apply(lambda x: " ".join(remove_punc(x).split()))
df

,class,word,sentence1,sentence2
0,NaN,kap,podaš komu na obodu ali ne sploh vas bo kap ko...,majico in kapo prejmejo
1,NaN,kap,skorumpiran preprodajalec orožja woe is us dež...,da sem prvič in zadnjič nosil peterokrako zvez...
2,NaN,kap,izdelki njegovih rok v rojstni hiši umrl je od...,s seboj krizopraz nesrečnežu ki ga je ohromila...
3,NaN,kap,odkrili kako lahko zaradi njih nastane srčni i...,čeprav je bilo zunaj borih trideset stopinj » ...
4,NaN,kap,to sploh nima veze ženska je res carica kapo dol,ne bi bilo v sobi “z dežja pod kap hm in nisi ...
...,...,...,...,...
4995,NaN,vila,minut vožnje od opatije je na ekskluzivni loka...,dolge kolone avtomobilov ki so se konec tedna ...
4996,NaN,vila,2 je pravljično lepa vila v resnici darkovo si...,iz katerega je molče snežilo se je razpenjalo ...
4997,NaN,vila,za najem stavbe npu hypo banko gradnjo vile na...,dokaj na hitro in brez možnosti podaljšanja po...
4998,NaN,vila,vila pod hollywoodskim znakom,gospa delopst pa da je prišla pred nje tudi z ...


In [4]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("w2v/model.bin", binary=True)

In [15]:
p1 = df.sentence1[100].split(" ")
p2 = df.sentence2[100].split(" ")
one = []
two = []
for i in range(len(p1)):
    try:
        one.append(model[p1[i]])
    except KeyError:
        pass
for i in range(len(p2)):
    try:
        two.append(model[p2[i]])
    except KeyError:
        pass

t1 = np.mean(one, axis=0)
t2 = np.mean(two, axis=0)
spatial.distance.cosine(t1, t2)

0.273398220539093

In [19]:
cls = []

for row in range(5000):
    p1 = df.sentence1[row].split(" ")
    p2 = df.sentence2[row].split(" ")
    one = []
    two = []
    for i in range(len(p1)):
        try:
            one.append(model[p1[i]])
        except KeyError:
            pass
    for i in range(len(p2)):
        try:
            two.append(model[p2[i]])
        except KeyError:
            pass
    t1 = np.mean(one, axis=0)
    t2 = np.mean(two, axis=0)
    res = spatial.distance.cosine(t1, t2)
    if res < 0.25:
        cls.append(1)
    else:
        cls.append(0)
print(len(cls))
df["class"] = cls
df

5000


,class,word,sentence1,sentence2
0,0,kap,podaš komu na obodu ali ne sploh vas bo kap ko...,majico in kapo prejmejo
1,0,kap,skorumpiran preprodajalec orožja woe is us dež...,da sem prvič in zadnjič nosil peterokrako zvez...
2,1,kap,izdelki njegovih rok v rojstni hiši umrl je od...,s seboj krizopraz nesrečnežu ki ga je ohromila...
3,1,kap,odkrili kako lahko zaradi njih nastane srčni i...,čeprav je bilo zunaj borih trideset stopinj » ...
4,1,kap,to sploh nima veze ženska je res carica kapo dol,ne bi bilo v sobi “z dežja pod kap hm in nisi ...
...,...,...,...,...
4995,0,vila,minut vožnje od opatije je na ekskluzivni loka...,dolge kolone avtomobilov ki so se konec tedna ...
4996,1,vila,2 je pravljično lepa vila v resnici darkovo si...,iz katerega je molče snežilo se je razpenjalo ...
4997,1,vila,za najem stavbe npu hypo banko gradnjo vile na...,dokaj na hitro in brez možnosti podaljšanja po...
4998,0,vila,vila pod hollywoodskim znakom,gospa delopst pa da je prišla pred nje tudi z ...


In [20]:
df["class"].value_counts()

1    2820
0    2180
Name: class, dtype: int64

In [21]:
df.to_csv("dataset_word2ve.csv")